In [1]:
from functions import *
from import_data import *

from astropy.constants import R_sun, L_sun, sigma_sb
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [2]:
# Import data
df_hmxb = HMXB_parameters()
df_falenga = falenga()
df_stellar_params = stellar_params()
df_photometric_params = photometric_params()
df_BJ = BailerJones()

<font color='yellow' size=5> Calculate observed luminosity from photometric filters </font>


In [3]:
df_L = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "L_true": [None] * len(df_hmxb['id']),
                              "L_true_err": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id']),
                              "logL_true/L_sun": [None] * len(df_hmxb['id']),
                              "logL_true/L_sun_err": [None] * len(df_hmxb["id"]),
                              "Teff": [None] * len(df_hmxb['id']),
                              "Teff_err": [None] * len(df_hmxb['id']),
                              "logTeff": [None] * len(df_hmxb['id']),
                              "logTeff_err": [None] * len(df_hmxb['id'])})

In [4]:
for i in range(df_hmxb.shape[0]):
    # Object id
    id = df_hmxb.loc[i, 'id']

    # Spectral type of object
    spectral_type = df_hmxb.loc[i, "ST"]

    # Effective temperature based on spectral type
    Teff = interpolate(df2=df_stellar_params, spectral_type=spectral_type, quantity='Teff')
    Teff_err = Teff_error(spectral_type)
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "Teff"] = Teff
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "Teff_err"] = Teff_err
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logTeff"] = np.log10(Teff)
    df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logTeff_err"] = Teff_err / (np.log(10) * Teff)

    if math.isnan(df_hmxb.loc[i, 'J']) == False and math.isnan(df_hmxb.loc[i, 'H']) == False and id in df_BJ['id'].tolist():
        # Expected (B-V)0 of object based on spectral type
        JH0 = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='(J-H)0')
        JH0_err = 0

        # Observed (B-V) of object based on simbad filters
        JHobs = df_hmxb.loc[i, 'J'] - df_hmxb.loc[i, 'H']
        JHobs_err = np.sqrt(0.03**2 + 0.03**2)

        # Bolometric correction (BC)
        BCh = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='BCh')

        # Distance
        distance = df_BJ.loc[df_BJ['id'] == id, 'r_med_photogeo'].reset_index(drop=True).at[0]
        distance_low = df_BJ.loc[df_BJ['id'] == id, 'r_lo_photogeo'].reset_index(drop=True).at[0]
        distance_high = df_BJ.loc[df_BJ['id'] == id, 'r_hi_photogeo'].reset_index(drop=True).at[0]
        d_err = ((distance - distance_low) + (distance_high - distance)) / 2

        # Calculate extinction
        Ah, Ah_err = extinction_and_error(0.46, 0.01, JHobs, JHobs_err, JH0, JH0_err)

        # Visual magnitude
        mh = df_hmxb.loc[i, 'H']

        # Calculate Absulute magnitude (visual)
        Mh = mh - 5 * np.log10(distance) + 5 - Ah

        # Calculate bolomatric absolute magnitude
        Mbol = Mh + BCh

        # Calculate the luminosity in solar luminosities
        L = 10**((Mbol - 4.74) / (-2.5))

        # Calculate the error on the luminosity
        L_err = luminosity_error(BCh, 0.1, mh, 0.03, distance, d_err, Ah, Ah_err)

        # Put L in luminosity dataframe
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err"] = L_err
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun"] = math.log10(L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err"] = L_err / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "ST"] = spectral_type



    if id == 'Cen X-3':
        # Bolometric correction (BC)
        BCv = interpolate(df2=df_photometric_params, spectral_type=spectral_type, quantity='BCv')

        # Distance
        distance = df_BJ.loc[df_BJ['id'] == id, 'r_med_photogeo'].reset_index(drop=True).at[0]
        distance_low = df_BJ.loc[df_BJ['id'] == id, 'r_lo_photogeo'].reset_index(drop=True).at[0]
        distance_high = df_BJ.loc[df_BJ['id'] == id, 'r_hi_photogeo'].reset_index(drop=True).at[0]
        d_err = ((distance - distance_low) + (distance_high - distance)) / 2

        # Calculate extinction
        Av = 3.2 * 1.4

        # Calculate Absulute magnitude (visual)
        # V=12.27 From INTEGRAL V band
        Mv = 12.27 - 5 * np.log10(distance) + 5 - Av

        # Calculate bolomatric absolute magnitude
        Mbol = Mv + BCv

        # Calculate the luminosity in solar luminosities
        L = 10**((Mbol - 4.74) / (-2.5))

        # Calculate error on luminosity
        L_err = luminosity_error(BCv, 0.1, 12.27, 0.03, distance, d_err, Av, 0.02)

        # Put L in luminosity dataframe
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true"] = L
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "L_true_err"] = L_err
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun"] = math.log10(L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "logL_true/L_sun_err"] = L_err / (np.log(10) * L)
        df_L.loc[df_L["id"] == df_hmxb.loc[i, 'id'], "ST"] = spectral_type

df_L = df_L.dropna(subset=['L_true']).reset_index(drop=True)

<font color='yellow' size=5>Calculate observed radius from observed luminosity and Teff from the models of Martins et al. (2005a) </font>

In [5]:
# Make radius dataframe
df_R = pd.DataFrame({'id': df_hmxb['id'].copy(),
                              "R_true": [None] * len(df_hmxb['id']),
                              "R_true_err": [None] * len(df_hmxb['id']),
                              "R_expected": [None] * len(df_hmxb['id']),
                              "R_expected_err": [None] * len(df_hmxb['id']),
                              "ST": [None] * len(df_hmxb['id'])})

In [6]:
for i in range(df_L.shape[0]):
    # id
    id = df_L.loc[i, 'id']

    # Spectral type
    spectral_type = df_L.loc[i, 'ST']

    # True luminosity
    L = df_L.loc[df_L["id"] == id, "L_true"].reset_index(drop=True).at[0]
    L_err = df_L.loc[df_L["id"] == id, "L_true_err"].reset_index(drop=True).at[0]
    
    # Effective temperature from model
    Teff = df_L.loc[df_L["id"] == id, "Teff"].reset_index(drop=True).at[0]
    Teff_err = df_L.loc[df_L["id"] == id, "Teff_err"].reset_index(drop=True).at[0]

    # Calculate the radius
    R, R_err = expected_radius_error(L, L_err, Teff, Teff_err)
    # R = math.sqrt((L_sun.value / R_sun.value**2) * (L / (4 * np.pi * sigma_sb.value * Teff**4)))
    # C = (4 * np.pi * sigma_sb.value * L_sun.value / R_sun.value**2)**(1/2)
    # R_err = ((C / ( 2 * L**(1/2) * Teff**2))**2 * L_err**2 + (2 * C * L**(1/2) / Teff**3)**2 * Teff_err**2)**(1/2)

    # Save luminosity in dataframe
    df_R.loc[df_R['id'] == id, "R_expected"] = R
    df_R.loc[df_R['id'] == id, "R_expected_err"] = R_err
    df_R.loc[df_R['id'] == id, "ST"] = spectral_type

<font color='yellow' size=5>Observed radius from Falenga et al. (2015) </font>

In [7]:
for i in range(df_falenga.shape[0]):
    # Object id
    id = df_falenga.loc[i, 'id']
    # Spectral type
    spectral_type = df_hmxb[df_hmxb["id"] == str(id)]["ST"].reset_index(drop=True).at[0]

    # Put R in luminosity dataframe
    df_R.loc[df_R["id"] == id, "R_true"] = df_falenga['R_opt'][i]
    df_R.loc[df_R["id"] == id, "R_true_err"] = df_falenga["R_opt_err"][i]
    df_R.loc[df_R["id"] == id, "ST"] = spectral_type

In [8]:
# Calculate ratio
df_R["R_expected/R_true"] = df_R["R_expected"] / df_R["R_true"]

In [9]:
df_R = df_R.dropna(subset=['R_true', 'R_expected']).reset_index(drop=True)

<font color='yellow' size=5>Mass </font>
- M_HRD is determined from the position in the HRD as shown below in the code
- M_opt is the determined mass from Falenga et al. (2015) based on the orbital parameters

In [10]:
df_M = pd.read_excel("tables/M_expBONNSAI.xlsx")
df_M = pd.merge(df_M, df_falenga, on='id')[["id", "M_HRD", "M_opt", "M_opt_err", "M_exp", "M_exp_err"]]
df_M["M_opt/M_exp"] = df_M["M_opt"] / df_M["M_exp"]

<font color='yellow' size=5>All parameters</font>

In [11]:
df_MRL = pd.merge(df_M, pd.merge(df_R, df_L, on='id'), on='id').drop(columns=['ST_y']).rename(columns={"ST_x": 'ST'})
df_MRL = df_MRL.drop(df_MRL.loc[df_MRL['id'] == 'OAO 1657-415'].index)

Calculate errors on M/M and R/R

In [12]:
df_MRL["M_opt/M_exp_err"] = ( (df_MRL["M_opt_err"] / df_MRL["M_opt"])**2 + (df_MRL["M_opt"] / (df_MRL["M_exp"])**2 * df_MRL["M_exp_err"] )**2 )**(1/2)
df_MRL["R_expected/R_true_err"] = ( (df_MRL["R_expected_err"] / df_MRL["R_expected"])**2 + (df_MRL["R_expected"] / (df_MRL["R_true"])**2 * df_MRL["R_true_err"])**2 )**(1/2)

In [13]:
df_MRL.to_csv('tables/results/MassRadiusLuminosity.csv', index=False)

In [14]:
df_MRL[["id", "logL_true/L_sun", "Teff", "R_expected"]]

,id,logL_true/L_sun,Teff,R_expected
0,SMC X-1,5.498888,29558.0,21.4164283825713
1,Vela X-1,5.598494,28653.0,25.5599607980095
2,LMC X-4,4.898484,33179.0,8.51468489012221
3,4U1700-37,5.700208,36801.0,17.4195866514551
4,4U1538-52,4.908964,29558.0,10.8588861484444
5,Cen X-3,5.766928,35895.0,19.7719494330861
6,XTE J1855-026,5.092961,29558.0,13.4209601267453
